# Entrenando un Modelo Markoviano Latente (HMM)

## Corpus de español:

* AnCora | Github: https://github.com/UniversalDependencies/UD_Spanish-AnCora

* usamos el conllu parser para leer el corpus: https://pypi.org/project/conllu/

* Etiquetas Universal POS (Documentación): https://universaldependencies.org/u/pos/

In [ ]:
#@title dependencias previas
!pip install conllu
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

Cloning into 'UD_Spanish-AnCora'...
remote: Enumerating objects: 1347, done.
remote: Counting objects: 100% (363/363), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 1347 (delta 282), reused 357 (delta 276), pack-reused 984
Receiving objects: 100% (1347/1347), 432.33 MiB | 19.03 MiB/s, done.
Resolving deltas: 100% (975/975), done.


In [ ]:
from conllu import parse_incr
from collections import defaultdict
import numpy as np
import nltk
from nltk import word_tokenize

In [ ]:
with open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8") as data_file:
    for tokenlist in parse_incr(data_file):
        print(tokenlist.serialize())

Se han truncado las últimas 5000 líneas del flujo de salida.
3	EJ10	EJ10	PROPN	_	_	2	flat	2:flat	_
4-5	del	_	_	_	_	_	_	_	_
4	de	de	ADP	spcms	_	6	case	6:case	_
5	el	el	DET	_	Definite=Def|Gender=Masc|Number=Sing|PronType=Art	6	det	6:det	_
6	alemán	alemán	NOUN	ncms000	Gender=Masc|Number=Sing	2	nmod	2:nmod	Entity=(CESSCASTAA2000062923978c27-person-1-gstype:spec
7	Heinz-Harald	Heinz-Harald	PROPN	np00000	_	6	appos	6:appos	MWE=Heinz-Harald_Frentzen|MWEPOS=PROPN|Entity=(NOCOREF:Spec.person-person-1-gstype:spec
8	Frentzen	Frentzen	PROPN	_	_	7	flat	7:flat	SpaceAfter=No|Entity=NOCOREF:Spec.person)
9	,	,	PUNCT	fc	PunctType=Comm	10	punct	10:punct	Entity=(CESSCASTAA2000062923978c27-person-2-CorefType:pred.definit,gstype:spec
10	vencedor	vencedor	NOUN	ncms000	Gender=Masc|Number=Sing	6	appos	6:appos	_
11	el	el	DET	da0ms0	Definite=Def|Gender=Masc|Number=Sing|PronType=Art	13	det	13:det	_
12	pasado	pasado	ADJ	aq0msp	Gender=Masc|Number=Sing|VerbForm=Part	13	amod	13:amod	_
13	año	año	NOUN	ncms000	Gender=Ma

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
#@title Estructura de los tokens etiquetados del corpus
tokenlist[1]

{'id': 2,
 'form': 'El',
 'lemma': 'el',
 'upos': 'DET',
 'xpos': 'da0ms0',
 'feats': {'Definite': 'Def',
  'Gender': 'Masc',
  'Number': 'Sing',
  'PronType': 'Art'},
 'head': 5,
 'deprel': 'det',
 'deps': [('det', 5)],
 'misc': {'Entity': '(NOCOREF:Gen--1-gstype:gen'}}

In [ ]:
tokenlist[1]['form']+'|'+tokenlist[1]['upos']

'El|DET'

## Entrenamiento del modelo - Calculo de conteos:

* tags (tags) `tagCountDict`: $C(tag)$
* emisiones (word|tag) `emissionProbDict`: $C(word|tag)$
* transiciones (tag|prevtag) `transitionDict`: $C(tag|prevtag)$

In [ ]:
tagCountDict = defaultdict(int)
emissionDict = defaultdict(int)
transitionDict = defaultdict(int)

tagtype = 'upos'
with open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8") as data_file:
    for tokenlist in parse_incr(data_file):
        prevtag = None
        for token in tokenlist:
            # C(tag)
            tag = token[tagtype]
            tagCountDict[tag] += 1

            # C(word|tag) -> probabilidades emision
            word_tag = f"{token['form'].lower()}|{token[tagtype]}" # (word|tag)
            emissionDict[word_tag] += 1

            # C(tag|tag_previo) -> probabilidades transición
            if prevtag is not None:
                transitiontags = f"{tag}|{prevtag}"
                transitionDict[transitiontags] += 1
            prevtag = tag

#transitionDict
#emissionDict
#tagCountDict

## Entrenamiento del modelo - calculo de probabilidades
* probabilidades de transición:
$$P(tag|prevtag) = \frac{C(prevtag, tag)}{C(prevtag)}$$

* probabilidades de emisión:
 $$P(word|tag) = \frac{C(word|tag)}{C(tag)}$$

In [ ]:
transitionProbDict = {}  # Matriz de transición (A)
emissionProbDict = {}    # Matriz de emisión (B)

# Probabilidades de transición
for key, value in transitionDict.items():
    # Dividir la etiqueta y la etiqueta previa
    tag, prevtag = key.split('|')
    if tagCountDict[prevtag] > 0:
        # Calcular la probabilidad de transición y almacenarla en transitionProbDict
        transitionProbDict[key] = value / tagCountDict[prevtag]
    else:
        # Imprimir las combinaciones de etiquetas que tienen un recuento previo de cero
        print("Recuento previo cero para:", key)

# Probabilidades de emisión
for key, value in emissionDict.items():
    # Dividir la palabra y la etiqueta
    word, tag = key.split('|')
    if tagCountDict[tag] > 0:
        # Calcular la probabilidad de emisión y almacenarla en emissionProbDict
        emissionProbDict[key] = value / tagCountDict[tag]
    else:
        # Imprimir las combinaciones de palabra y etiqueta que tienen un recuento de etiqueta cero
        print("Recuento de etiqueta cero para:", key)

# Ejemplo de acceso a la probabilidad de transición para la etiqueta 'ADJ' dado que la etiqueta anterior es 'ADJ'
transitionProbDict['ADJ|ADJ']
# emissionProbDict

0.030217452696978255

## Guardar parámetros del modelo

In [ ]:
# Guardar los diccionarios de probabilidades de transición y emisión como archivos numpy
np.save('transitionHMM.npy', transitionProbDict)
np.save('emissionHMM.npy', emissionProbDict)

In [ ]:
# Cargar los diccionarios de probabilidades de transición y emisión desde los archivos numpy
transitionProbDict = np.load('transitionHMM.npy', allow_pickle=True).item()
emissionProbDict = np.load('emissionHMM.npy', allow_pickle=True).item()

In [ ]:
# Ejemplo de acceso a la probabilidad de transición para la etiqueta 'ADJ' dado que la etiqueta anterior es 'ADJ'
transitionProbDict['ADJ|ADJ']

0.030217452696978255

# Carga del modelo HMM previamente entrenado

In [ ]:
# cargamos las probabilidades del modelo HMM
transitionProbdict = np.load('transitionHMM.npy', allow_pickle='TRUE').item()
emissionProbdict = np.load('emissionHMM.npy', allow_pickle='TRUE').item()

In [ ]:
# Identificamos las categorías gramaticales 'upos' únicas en el corpus

stateSet = set()

for key in emissionProbDict.keys():
  stateSet.add(key.split('|')[1])

stateSet

{'ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CCONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'PUNCT',
 'SCONJ',
 'SYM',
 'VERB',
 '_'}

In [ ]:
# enumeramos las categorias con numeros para asignar a
# las columnas de la matriz de Viterbi

tagStateDict = {}

for i, state in enumerate(stateSet):
  tagStateDict[state] = i

tagStateDict

{'CCONJ': 0,
 'ADJ': 1,
 'DET': 2,
 'NOUN': 3,
 'PRON': 4,
 'PUNCT': 5,
 '_': 6,
 'PROPN': 7,
 'NUM': 8,
 'PART': 9,
 'ADV': 10,
 'SYM': 11,
 'AUX': 12,
 'VERB': 13,
 'ADP': 14,
 'SCONJ': 15,
 'INTJ': 16}

In [ ]:
lista = [item for item in tagStateDict.items()]

In [ ]:
lista

[('CCONJ', 0),
 ('ADJ', 1),
 ('DET', 2),
 ('NOUN', 3),
 ('PRON', 4),
 ('PUNCT', 5),
 ('_', 6),
 ('PROPN', 7),
 ('NUM', 8),
 ('PART', 9),
 ('ADV', 10),
 ('SYM', 11),
 ('AUX', 12),
 ('VERB', 13),
 ('ADP', 14),
 ('SCONJ', 15),
 ('INTJ', 16)]

# Distribucion inicial de estados latentes

In [ ]:
# Calculamos distribución inicial de estados
initTagStateProb = {}  # \rho_i^{(0)}

# Inicializamos una lista vacía para almacenar las palabras del corpus
wordList = []

# Abrimos el archivo del corpus para lectura
with open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8") as data_file:
    count = 0  # Contador para la longitud del corpus

    # Iteramos sobre cada oración del corpus
    for tokenlist in parse_incr(data_file):
        count += 1
        # Obtenemos la etiqueta gramatical de la primera palabra de la oración
        tag = tokenlist[0]['upos']

        # Contamos la frecuencia de cada etiqueta gramatical como estado inicial
        initTagStateProb[tag] = initTagStateProb.get(tag, 0) + 1

# Normalizamos las frecuencias para obtener probabilidades
for key in initTagStateProb:
    initTagStateProb[key] /= count


In [ ]:
# verificamos que la suma de las probabilidades es 1 (100%)
np.array([initTagStateProb[k] for k in initTagStateProb.keys()]).sum()

1.0

# Construcción del algoritmo de Viterbi






Dada una secuencia de palabras $\{p_1, p_2, \dots, p_n \}$, y un conjunto de categorias gramaticales dadas por la convención `upos`, se considera la matriz de probabilidades de Viterbi así:

$$
\begin{array}{c c}
\begin{array}{c c c c}
\text{ADJ} \\
\text{ADV}\\
\text{PRON} \\
\vdots \\
{}
\end{array}
&
\left[
\begin{array}{c c c c}
\nu_1(\text{ADJ}) & \nu_2(\text{ADJ}) & \dots  & \nu_n(\text{ADJ})\\
\nu_1(\text{ADV}) & \nu_2(\text{ADV}) & \dots  & \nu_n(\text{ADV})\\
\nu_1(\text{PRON}) & \nu_2(\text{PRON}) & \dots  & \nu_n(\text{PRON})\\
\vdots & \vdots & \dots & \vdots \\ \hdashline
p_1 & p_2 & \dots & p_n
\end{array}
\right]
\end{array}
$$

Donde las probabilidades de la primera columna (para una categoria $i$) están dadas por:

$$
\nu_1(i) = \underbrace{\rho_i^{(0)}}_{\text{probabilidad inicial}} \times \underbrace{P(p_1 \vert i)}_{\text{emisión}}
$$

luego, para la segunda columna (dada una categoria $j$) serán:

$$
\nu_2(j) = \max_i \{ \nu_1(i) \times \underbrace{P(j \vert i)}_{\text{transición}} \times \underbrace{P(p_2 \vert j)}_{\text{emisión}} \}
$$

así, en general las probabilidades para la columna $t$ estarán dadas por:

$$
\nu_{t}(j) = \max_i \{ \overbrace{\nu_{t-1}(i)}^{\text{estado anterior}} \times \underbrace{P(j \vert i)}_{\text{transición}} \times \underbrace{P(p_t \vert j)}_{\text{emisión}} \}
$$

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def ViterbiMatrix(secuencia, transitionProbDict=transitionProbDict, emissionProbDict=emissionProbDict,
                  tagStateDict=tagStateDict, initTagStateProb=initTagStateProb):
    """
    Calcula la matriz Viterbi para una secuencia dada.

    Parámetros:
        secuencia (str): La secuencia de palabras para la cual se calculará la matriz Viterbi.
        transitionProbDict (dict): Diccionario que contiene las probabilidades de transición.
        emissionProbDict (dict): Diccionario que contiene las probabilidades de emisión.
        tagStateDict (dict): Diccionario que mapea las etiquetas de estado a filas de la matriz Viterbi.
        initTagStateProb (dict): Diccionario que contiene las probabilidades iniciales de los estados.

    Retorna:
        numpy.ndarray: La matriz Viterbi calculada.
    """
    # Tokenizar la secuencia de entrada
    seq = word_tokenize(secuencia)
    num_tags = len(tagStateDict)
    viterbiProb = np.zeros((num_tags, len(seq)))  # upos tiene 17 categorías

    # Inicialización primera columna
    for key, tag_row in tagStateDict.items():
        word_tag = f"{seq[0].lower()}|{key}"
        if word_tag in emissionProbDict:
            viterbiProb[tag_row, 0] = initTagStateProb[key] * emissionProbDict[word_tag]

    # Cálculo de las siguientes columnas
    for col in range(1, len(seq)):
        for key, tag_row in tagStateDict.items():
            word_tag = f"{seq[col].lower()}|{key}"
            if word_tag in emissionProbDict:
                # Miramos estados de la columna anterior
                possible_probs = [
                    viterbiProb[tag_row2, col - 1] * transitionProbDict[f"{key2}|{key}"] * emissionProbDict[word_tag]
                    for key2, tag_row2 in tagStateDict.items()
                    if f"{key2}|{key}" in transitionProbDict and viterbiProb[tag_row2, col - 1] > 0
                ]
                viterbiProb[tag_row, col] = max(possible_probs, default=0)

    return viterbiProb

# Ejemplo de uso
matrix = ViterbiMatrix('el mundo es pequeño')
matrix


array([[0.00000000e+00, 0.00000000e+00, 3.19336137e-10, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 5.40681635e-13],
       [1.24339097e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 4.04259663e-06, 1.23362810e-11, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.66211053e-06, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 2.17289569e-08, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e

In [ ]:
def ViterbiTags(secuencia, transitionProbdict=transitionProbdict, emissionProbdict=emissionProbdict,
            tagStateDict=tagStateDict, initTagStateProb=initTagStateProb):
  seq = word_tokenize(secuencia)
  viterbiProb = np.zeros((17, len(seq)))  # upos tiene 17 categorias

  # inicialización primera columna
  for key in tagStateDict.keys():
    tag_row = tagStateDict[key]
    word_tag = seq[0].lower()+'|'+key
    if word_tag in emissionProbdict.keys():
      viterbiProb[tag_row, 0] = initTagStateProb[key]*emissionProbdict[word_tag]

  # computo de las siguientes columnas
  for col in range(1, len(seq)):
    for key in tagStateDict.keys():
      tag_row = tagStateDict[key]
      word_tag = seq[col].lower()+'|'+key
      if word_tag in emissionProbdict.keys():
        # miramos estados de la col anterior
        possible_probs = []
        for key2 in tagStateDict.keys():
          tag_row2 = tagStateDict[key2]
          tag_prevtag = key+'|'+key2
          if tag_prevtag in transitionProbdict.keys():
            if viterbiProb[tag_row2, col-1]>0:
              possible_probs.append(
                  viterbiProb[tag_row2, col-1]*transitionProbdict[tag_prevtag]*emissionProbdict[word_tag])
        viterbiProb[tag_row, col] = max(possible_probs)

    # contruccion de secuencia de tags
    res = []
    for i, p in enumerate(seq):
      for tag in tagStateDict.keys():
        if tagStateDict[tag] == np.argmax(viterbiProb[:, i]):
          res.append((p, tag))

  return res

ViterbiTags('el mundo es muy pequeño')

[('el', 'DET'),
 ('mundo', 'NOUN'),
 ('es', 'AUX'),
 ('muy', 'ADV'),
 ('pequeño', 'ADJ')]

In [ ]:
ViterbiTags('estos instrumentos han de rasgar')

[('estos', 'DET'),
 ('instrumentos', 'NOUN'),
 ('han', 'AUX'),
 ('de', 'ADP'),
 ('rasgar', 'VERB')]

# Entrenamiento directo de HMM con NLTK

* clase en python (NLTK) de HMM: https://www.nltk.org/_modules/nltk/tag/hmm.html

In [ ]:
#@title ejemplo con el Corpus Treebank en ingles
import nltk
nltk.download('treebank')
from nltk.corpus import treebank
train_data = treebank.tagged_sents()[:3900]

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


In [ ]:
#@title estructura de la data de entrenamiento
train_data

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], ...]

In [ ]:
#@title HMM pre-construido en NLTK
from nltk.tag import hmm
tagger = hmm.HiddenMarkovModelTrainer().train_supervised(train_data)
tagger

<HiddenMarkovModelTagger 46 states and 12385 output symbols>

In [ ]:
tagger.tag("Pierre Vinken will get old".split())

/usr/local/lib/python3.10/dist-packages/nltk/tag/hmm.py:334: RuntimeWarning: overflow encountered in cast
  X[i, j] = self._transitions[si].logprob(self._states[j])
/usr/local/lib/python3.10/dist-packages/nltk/tag/hmm.py:336: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])
/usr/local/lib/python3.10/dist-packages/nltk/tag/hmm.py:332: RuntimeWarning: overflow encountered in cast
  P[i] = self._priors.logprob(si)


[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 ('will', 'MD'),
 ('get', 'VB'),
 ('old', 'JJ')]

In [ ]:
#@title training accuracy
tagger.evaluate(treebank.tagged_sents()[:3900])

<ipython-input-26-36c901df877a>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  tagger.evaluate(treebank.tagged_sents()[:3900])


0.9815403947224078

## Ejercicio de práctica

**Objetivo:** Entrena un HMM usando la clase `hmm.HiddenMarkovModelTrainer()` sobre el dataset `UD_Spanish_AnCora`.

1. **Pre-procesamiento:** En el ejemplo anterior usamos el dataset en ingles `treebank`, el cual viene con una estructura diferente a la de `AnCora`, en esta parte escribe código para transformar la estructura de `AnCora` de manera que quede igual al `treebank` que usamos así:

$$\left[ \left[ (\text{'El'}, \text{'DET'}), (\dots), \dots\right], \left[\dots \right] \right]$$

In [ ]:
# desarrolla tu código aquí


2. **Entrenamiento:** Una vez que el dataset esta con la estructura correcta, utiliza la clase `hmm.HiddenMarkovModelTrainer()` para entrenar con el $80 \%$ del dataset como conjunto de `entrenamiento` y $20 \%$ para el conjunto de `test`.

**Ayuda:** Para la separacion entre conjuntos de entrenamiento y test, puedes usar la funcion de Scikit Learn:

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

En este punto el curso de Machine Learning con Scikit Learn es un buen complemento para entender mejor las funcionalidades de Scikit Learn: https://platzi.com/cursos/scikitlearn-ml/

In [ ]:
# desarrolla tu código aquí


3. **Validación del modelo:** Un vez entrenado el `tagger`, calcula el rendimiento del modelo (usando `tagger.evaluate()`) para los conjuntos de `entrenamiento` y `test`.



In [ ]:
#desarrolla tu código aquí


## Construcción de un modelo markoviano de máxima entropía


In [ ]:
!pip install conllu
!pip install stanza
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.1/990.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 11.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

### Entrenamiento del modelo - cálculo de conteos

Parta este modelo consideramos el cálculo de las probabilidades:

$$P(t_i | w_i, t_{i-1}) =\frac{C(w_i, t_i, t_{i-1})}{C(w_i, t_{i-1})} $$

* `uniqueFeatureDict` $C(tag|word,prevtag) = C(w_i, t_i, t_{i-1})$
* `contextDict` $C(word,prevtag) = C(w_i, t_{i-1})$

En este caso cuando consideremos el primer elemento de una frase $w_0$, no existirá un elemento anterior $w_{-1}$ y por lo tanto, tampoco una etiqueta previa $t_{-1}$, podemos modelar este problema asumiendo que existe una etiqueta "None", para estos casos:

$$P(t_0|w_0,t_{-1}) = P(t_0|w_0,\text{"None"})$$

In [ ]:
from conllu import parse_incr

uniqueFeatureDict = {}
contextDict = {}

tagtype = 'upos'
data_file = open("UD_Spanish-AnCora/es_ancora-ud-train.conllu", "r", encoding="utf-8")

# Calculando conteos (pre-probabilidades)
for tokenlist in parse_incr(data_file):
  prevtag = "None"
  for token in tokenlist:
    tag = token[tagtype]
    word = token['form'].lower()
    #C(tag|word,prevtag)
    largeKey = tag+'|'+word+','+prevtag
    if largeKey in uniqueFeatureDict.keys():
      uniqueFeatureDict[largeKey]+=1
    else:
      uniqueFeatureDict[largeKey]=1
    key = word+','+prevtag
    if key in contextDict.keys():
      contextDict[key]+=1
    else:
      contextDict[key]=1
    #print(largeKey, key, '\n')
    prevtag=tag

### Entrenamiento del modelo - cálculo de probabilidades

$$P(t_i|w_i, t_{i-1}) = \frac{C(t_i, w_i, t_{i-1})}{C(w_i, t_{i-1})}$$

In [ ]:
posteriorProbDict = {}

for key in uniqueFeatureDict.keys():
  if len(key.split('|'))==3:
    posteriorProbDict[key] = uniqueFeatureDict[key]/contextDict['|'+key.split('|')[-1]]
  else:
    posteriorProbDict[key] = uniqueFeatureDict[key]/contextDict[key.split('|')[1]]

In [ ]:
# Aquí verificamos que todas las probabilidades
# por cada contexto 'word,prevtag' suman 1.0

for base_context in contextDict.keys():
  countprob = 0
  items = 0
  for key in posteriorProbDict.keys():
    if len(key.split('|'))==3:
      if '|'+key.split('|')[-1]==base_context:
        countprob+=posteriorProbDict[key]
        items+=1
    else:
      if key.split('|')[1]==base_context:
        countprob+=posteriorProbDict[key]
        items+=1
  print(base_context, items, countprob)

### Distribución inicial de estados latentes

In [ ]:
# identificamos las categorias gramaticales 'upos' unicas en el corpus
stateSet = {'ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CCONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'PUNCT',
 'SCONJ',
 'SYM',
 'VERB',
 '_'}
# enumeramos las categorias con numeros para asignar a
# las columnas de la matriz de Viterbi
tagStateDict = {}
for i, state in enumerate(stateSet):
  tagStateDict[state] = i
tagStateDict

{'ADJ': 9,
 'ADP': 8,
 'ADV': 1,
 'AUX': 14,
 'CCONJ': 4,
 'DET': 12,
 'INTJ': 15,
 'NOUN': 2,
 'NUM': 7,
 'PART': 3,
 'PRON': 5,
 'PROPN': 0,
 'PUNCT': 10,
 'SCONJ': 6,
 'SYM': 16,
 'VERB': 11,
 '_': 13}

In [ ]:
initTagStateProb = {} # \rho_i^{(0)}
from conllu import parse_incr
wordList = []
data_file = open("UD_Spanish-AnCora/es_ancora-ud-train.conllu", "r", encoding="utf-8")
count = 0 # cuenta la longitud del corpus
for tokenlist in parse_incr(data_file):
  count += 1
  tag = tokenlist[0]['upos']
  if tag in initTagStateProb.keys():
    initTagStateProb[tag] += 1
  else:
    initTagStateProb[tag] = 1

for key in initTagStateProb.keys():
  initTagStateProb[key] /= count

initTagStateProb

{'ADJ': 0.010136315973435861,
 'ADP': 0.1574274729115694,
 'ADV': 0.07577770010485844,
 'AUX': 0.022789234533379936,
 'CCONJ': 0.036980076896190144,
 'DET': 0.34799021321216356,
 'INTJ': 0.0020272631946871723,
 'NOUN': 0.025026214610276126,
 'NUM': 0.0068507514854945824,
 'PART': 0.002446696959105208,
 'PRON': 0.04173365955959455,
 'PROPN': 0.10506815798671792,
 'PUNCT': 0.09143656064313177,
 'SCONJ': 0.027123383432366307,
 'SYM': 0.0004893393918210416,
 'VERB': 0.04557846906675987,
 '_': 0.0011184900384480952}

### Construcción del algoritmo de Viterbi

Dada una secuencia de palabras $\{p_1, p_2, \dots, p_n \}$, y un conjunto de categorias gramaticales dadas por la convención `upos`, se considera la matriz de probabilidades de Viterbi así:

$$
\begin{array}{c c}
\begin{array}{c c c c}
\text{ADJ} \\
\text{ADV}\\
\text{PRON} \\
\vdots \\
{}
\end{array}
&
\left[
\begin{array}{c c c c}
\nu_1(\text{ADJ}) & \nu_2(\text{ADJ}) & \dots  & \nu_n(\text{ADJ})\\
\nu_1(\text{ADV}) & \nu_2(\text{ADV}) & \dots  & \nu_n(\text{ADV})\\
\nu_1(\text{PRON}) & \nu_2(\text{PRON}) & \dots  & \nu_n(\text{PRON})\\
\vdots & \vdots & \dots & \vdots \\ \hdashline
p_1 & p_2 & \dots & p_n
\end{array}
\right]
\end{array}
$$

Donde las probabilidades de Viterbi en la primera columna (para una categoria $i$) están dadas por:

$$
\nu_1(i) = \underbrace{\rho_i^{(0)}}_{\text{probabilidad inicial}} \times P(i \vert p_1, \text{"None"})
$$

y para las siguientes columnas:

$$
\nu_{t}(j) = \max_i \{ \overbrace{\nu_{t-1}(i)}^{\text{estado anterior}} \times P(j \vert p_t, i) \}
$$


In [ ]:
import numpy as np
import stanza
stanza.download('es')
nlp = stanza.Pipeline('es', processors='tokenize')

2020-08-18 02:14:23 INFO: Downloading default packages for language: es (Spanish)...
2020-08-18 02:14:25 INFO: File exists: /root/stanza_resources/es/default.zip.
2020-08-18 02:14:31 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
def ViterbiMatrix(secuencia, posteriorProbDict=posteriorProbDict, initTagStateProb=initTagStateProb):
  doc = nlp(secuencia)
  if len(doc.sentences)>1:
    raise ValueError('secuencia must be a string!')
  seq = [word.text for word in doc.sentences[0].words]
  viterbiProb = np.zeros((17, len(seq)))

  # inicialización primera columna
  for tag in tagStateDict.keys():
    tag_row = tagStateDict[tag]
    key = tag+'|'+seq[0].lower()+','+"None"
    try:
      viterbiProb[tag_row, 0] = initTagStateProb[tag]*posteriorProbDict[key]
    except:
      pass

  # computo de las siguientes columnas
  for col in range(1, len(seq)):
    for tag in tagStateDict.keys():
      tag_row = tagStateDict[tag]
      possible_probs = []
      for prevtag in tagStateDict.keys():
        prevtag_row = tagStateDict[prevtag]
        key = tag+'|'+seq[col].lower()+','+prevtag
        try:
          possible_probs.append(
              viterbiProb[prevtag_row, col-1]*posteriorProbDict[key])
        except:
          possible_probs.append(0)
      viterbiProb[tag_row, col] = max(possible_probs)

  return viterbiProb

ViterbiMatrix('el mundo es pequeño')

array([[0.00000000e+00, 8.22024126e-03, 1.13643888e-04, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.39769972e-01, 5.94927966e-03, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.33820692e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.47990213e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e

In [ ]:
def ViterbiTags(secuencia, posteriorProbDict=posteriorProbDict, initTagStateProb=initTagStateProb):
  doc = nlp(secuencia)
  if len(doc.sentences)>1:
    raise ValueError('secuencia must be a string!')
  seq = [word.text for word in doc.sentences[0].words]
  viterbiProb = np.zeros((17, len(seq)))

  # inicialización primera columna
  for tag in tagStateDict.keys():
    tag_row = tagStateDict[tag]
    key = tag+'|'+seq[0].lower()+','+"None"
    try:
      viterbiProb[tag_row, 0] = initTagStateProb[tag]*posteriorProbDict[key]
    except:
      pass


  # computo de las siguientes columnas
  for col in range(1, len(seq)):
    for tag in tagStateDict.keys():
      tag_row = tagStateDict[tag]
      possible_probs = []
      for prevtag in tagStateDict.keys():
        prevtag_row = tagStateDict[prevtag]
        key = tag+'|'+seq[col].lower()+','+prevtag
        try:
          possible_probs.append(
              viterbiProb[prevtag_row, col-1]*posteriorProbDict[key])
        except:
          possible_probs.append(0)
      viterbiProb[tag_row, col] = max(possible_probs)

  # contruccion de secuencia de tags
  res = []
  for i, p in enumerate(seq):
    for tag in tagStateDict.keys():
      if tagStateDict[tag] == np.argmax(viterbiProb[:, i]):
        res.append((p, tag))


  return res

ViterbiTags('el mundo es pequeño')

[('el', 'DET'), ('mundo', 'NOUN'), ('es', 'AUX'), ('pequeño', 'ADJ')]

In [ ]:
ViterbiTags('estos instrumentos han de rasgar')

[('estos', 'DET'),
 ('instrumentos', 'NOUN'),
 ('han', 'AUX'),
 ('de', 'ADP'),
 ('rasgar', 'PROPN')]

## ¿ Siguientes Pasos ?

El modelo construido, aunque es la base de un MEMM, no explota todo el potencial del concepto  que estos modelos representan, en nuestro caso sencillo consideramos solo un **feature** para predecir la categoría gramatical: $<w_i, t_{i-1}>$. Es decir, las probabilidades de una cierta etiqueta $t_i$ dada una observación $<w_i, t_{i-1}>$ se calculan contando eventos donde se observe que $<w_i, t_{i-1}>$ sucede simultáneamente con $t_i$.

La generalización de esto (donde puedo considerar multiples observaciones o **features**, y a partir de estos inferir la categoría gramatical) se hace construyendo las llamadas **feature-functions**, donde estas funciones toman valores de 0 o 1, cuando se cumplan las condiciones de la observación o feature en cuestion. En general podemos considerar una **feature-function** como :

$$f_a(t, o) = f_a(\text{tag}, \text{observation}) =
\begin{cases}
  1 , & \text{se cumple condición } a \\
  0, & \text{en caso contrario}
\end{cases}
$$

donde la condición $a$ es una relacion entre los valores que tome $\text{tag}$ y $\text{context}$, por ejemplo:

$$f_a(t, o) = f_a(\text{tag}, \text{observation}) =
\begin{cases}
  1 , & (t_i, t_{i-1}) = \text{('VERB', 'ADJ')} \\
  0, & \text{en caso contrario}
\end{cases}
$$

Al considerar varias funciones, y por lo tanto varios features observables, consideramos una combinacion lineal de estos por medio de un coeficiente que multiplique a cada función:

$$
\theta_1 f_1(t, o) + \theta_2 f_2(t, o) + \dots
$$

donde los coeficientes indicarán cuales features son más relevantes y por lo tanto pesan más para la decisión del resultado del modelo. De esta manera los coeficientes $\theta_j$ se vuelven parámetros del modelo que deben ser optimizados (esto puede realizarse con cualquier técnica de optimizacion como el Gradiente Descendente). Ahora, las probabilidades que pueden obtener usando un softmax sobre estas combinaciones lineales de features:

$$
P = \prod_i \frac{\exp{\left(\sum_j \theta_j f_j(t_i, o)\right)}}{\sum_{t'}\exp{\left(\sum_j \theta_j f_j(t', o)\right)}}
$$

Así, lo que buscamos con el algoritmo de optimización es encontrar los parámetros $\theta_j$ que maximizan la probabilidad anterior. En NLTK encontramos la implementación completa de un clasificador de máxima entropia que no esta restringido a relaciones markovianas: https://www.nltk.org/_modules/nltk/classify/maxent.html

Un segmento resumido de la clase en python que implementa este clasificador en NLTK lo encuentras así:

```
class MaxentClassifier(ClassifierI):

    def __init__(self, encoding, weights, logarithmic=True):
        self._encoding = encoding
        self._weights = weights
        self._logarithmic = logarithmic
        assert encoding.length() == len(weights)

    def labels(self):
        return self._encoding.labels()

    def set_weights(self, new_weights):
        self._weights = new_weights
        assert self._encoding.length() == len(new_weights)


    def weights(self):
        return self._weights

    def classify(self, featureset):
        return self.prob_classify(featureset).max()

    def prob_classify(self, featureset):
        ### ...

        # Normalize the dictionary to give a probability distribution
        return DictionaryProbDist(prob_dict, log=self._logarithmic, normalize=True)

    @classmethod
    def train(
        cls,
        train_toks,
        algorithm=None,
        trace=3,
        encoding=None,
        labels=None,
        gaussian_prior_sigma=0,
        **cutoffs
    ):
     ### ......
```

Donde te das cuenta de la forma que tienen las clases en NLTK que implementan clasificadores generales. Aquí vemos que la clase `MaxentClassifier` es una subclase de una más general `ClassifierI` la cual representa el proceso de clasificación general de categoría única (es decir, que a cada data-point le corresponda solo una categoria), también que esta clase depende de definir un `encoding`
 y unos pesos `weights` :

```
class MaxentClassifier(ClassifierI):

    def __init__(self, encoding, weights, logarithmic=True):
```

los pesos corresponden a los parámetros $\theta_i$. Y el encoding es el que corresponde a las funciones $f_a(t, o)$ que dan como resultado valores binarios $1$ o $0$.

La documentación de NLTK te puede dar mas detalles de esta implementación: https://www.nltk.org/api/nltk.classify.html

Finalmente, un ejemplo completo de uso y mejora de un modelo de máxima entropía, lo puedes encontrar en este fork que guarde especialmente para el curso, para que lo tengas de referencia y puedas jugar y aprender con él:

https://github.com/pachocamacho1990/nltk-maxent-pos-tagger

El cual fue desarrollado originalmente por Arne Neumann (https://github.com/arne-cl) basado en los fueatures propuestos por Ratnaparki en 1996 para la tarea de etiquetado por categorias gramaticales.


# Clasificación de palabras (por género de nombre)

In [ ]:
import nltk, random
from nltk.corpus import names

In [ ]:
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.


True

**Función básica de extracción de atributos**

In [ ]:
# definición de atributos relevantes
def atributos(palabra):
	return {'ultima_letra': palabra[-1]}

tagset = ([(name, 'male') for name in names.words('male.txt')] + [(name, 'female') for name in names.words('female.txt')])

In [ ]:
tagset[:10]

[('Aamir', 'male'),
 ('Aaron', 'male'),
 ('Abbey', 'male'),
 ('Abbie', 'male'),
 ('Abbot', 'male'),
 ('Abbott', 'male'),
 ('Abby', 'male'),
 ('Abdel', 'male'),
 ('Abdul', 'male'),
 ('Abdulkarim', 'male')]

In [ ]:
random.shuffle(tagset)
tagset[:10]

[('Armond', 'male'),
 ('Leodora', 'female'),
 ('Webster', 'male'),
 ('Ashlen', 'female'),
 ('Talbert', 'male'),
 ('Reeva', 'female'),
 ('Linzy', 'female'),
 ('Veronique', 'female'),
 ('Fredrika', 'female'),
 ('Earle', 'male')]

In [ ]:
fset = [(atributos(n), g) for (n, g) in tagset]
train, test = fset[500:], fset[:500]

**Modelo de clasificación Naive Bayes**

In [ ]:
# entrenamiento del modelo NaiveBayes
classifier = nltk.NaiveBayesClassifier.train(train)

 **Verificación de algunas predicciones**

In [ ]:
classifier.classify(atributos('amanda'))

'female'

In [ ]:
classifier.classify(atributos('peter'))

'male'

**Performance del modelo**

In [ ]:
print(nltk.classify.accuracy(classifier, test))

0.734


In [ ]:
print(nltk.classify.accuracy(classifier, train))

0.7647770016120365


**Mejores atributos**

In [ ]:
def mas_atributos(nombre):
    atrib = {}
    atrib["primera_letra"] = nombre[0].lower()
    atrib["ultima_letra"] = nombre[-1].lower()
    for letra in 'abcdefghijklmnopqrstuvwxyz':
        atrib["count({})".format(letra)] = nombre.lower().count(letra)
        atrib["has({})".format(letra)] = (letra in nombre.lower())
    return atrib

In [ ]:
mas_atributos('jhon')

In [ ]:
fset = [(mas_atributos(n), g) for (n, g) in tagset]
train, test = fset[500:], fset[:500]
classifier2 = nltk.NaiveBayesClassifier.train(train)

In [ ]:
print(nltk.classify.accuracy(classifier2, test))

0.77


### Ejercicio de práctica

**Objetivo:** Construye un classificador de nombres en español usando el siguiente dataset:
https://github.com/jvalhondo/spanish-names-surnames

1. **Preparación de los datos**: con un `git clone` puedes traer el dataset indicado a tu directorio en Colab, luego asegurate de darle el formato adecuado a los datos y sus features para que tenga la misma estructura del ejemplo anterior con el dataset `names` de nombres en ingles.

* **Piensa y analiza**: ¿los features en ingles aplican de la misma manera para los nombres en español?

In [ ]:
# escribe tu código aquí


2. **Entrenamiento y performance del modelo**: usando el classificador de Naive Bayes de NLTK entrena un modelo sencillo usando el mismo feature de la última letra del nombre, prueba algunas predicciones y calcula el performance del modelo.

In [ ]:
# escribe tu código aquí


3. **Mejores atributos:** Define una función como `atributos2()` donde puedas extraer mejores atributos con los cuales entrenar una mejor version del clasificador. Haz un segundo entrenamiento y verifica como mejora el performance de tu modelo. ¿Se te ocurren mejores maneras de definir atributos para esta tarea particular?

In [ ]:
# escribe tu código aquí


# Clasificación de documentos (email spam o no spam)

In [ ]:
!git clone https://github.com/pachocamacho1990/datasets

fatal: destination path 'datasets' already exists and is not an empty directory.


In [ ]:
import pandas as pd
import numpy as np
from nltk import word_tokenize

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
df = pd.read_csv('datasets/email/csv/spam-apache.csv', names = ['clase','contenido'])
df['tokens'] = df['contenido'].apply(lambda x: word_tokenize(x))
df.head()

,clase,contenido,tokens
0,-1,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...","[<, !, DOCTYPE, HTML, PUBLIC, ``, -//W3C//DTD,..."
1,1,> Russell Turpin:\n> > That depends on how the...,"[>, Russell, Turpin, :, >, >, That, depends, o..."
2,-1,Help wanted. We are a 14 year old fortune 500...,"[Help, wanted, ., We, are, a, 14, year, old, f..."
3,-1,Request A Free No Obligation Consultation!\nAc...,"[Request, A, Free, No, Obligation, Consultatio..."
4,1,Is there a way to look for a particular file o...,"[Is, there, a, way, to, look, for, a, particul..."


In [ ]:
df['tokens'].values[0]

In [ ]:
all_words = nltk.FreqDist([w for tokenlist in df['tokens'].values for w in tokenlist])
top_words = all_words.most_common(200)

def document_features(document):
    document_words = set(document)
    features = {}
    for word in top_words:
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [ ]:
document_features(df['tokens'].values[0])

In [ ]:
limit = 200
fset = [(document_features(texto), clase) for texto, clase in zip(df['tokens'].values, df['clase'].values)]
random.shuffle(fset)
train, test = fset[:limit], fset[limit:]

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train)

In [ ]:
print(nltk.classify.accuracy(classifier, test))

0.48


In [ ]:
classifier.show_most_informative_features(5)

Most Informative Features
    contains(('us', 53)) = False               1 : -1     =      1.0 : 1.0
 contains(('could', 66)) = False               1 : -1     =      1.0 : 1.0
 contains(('money', 72)) = False               1 : -1     =      1.0 : 1.0
  contains(('days', 40)) = False               1 : -1     =      1.0 : 1.0
  contains(('need', 43)) = False               1 : -1     =      1.0 : 1.0


In [ ]:
df[df['clase']==-1]['contenido']

0      <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.0 Tr...
2      Help wanted.  We are a 14 year old fortune 500...
3      Request A Free No Obligation Consultation!\nAc...
10     >\n>“µ×è¹µÑÇ ¡ÑºâÅ¡¸ØÃ¡Ô¨º¹ÍÔ¹àµÍÃìà¹çµ” \n>àµ...
11     ==============================================...
                             ...                        
243    ##############################################...
244    Wanna see sexually curious teens playing with ...
246    REQUEST FOR URGENT BUSINESS ASSISTANCE\n------...
248    Email marketing works!  There's no way around ...
249    Email marketing works!  There's no way around ...
Name: contenido, Length: 125, dtype: object

## Ejercicio de práctica


¿Como podrías construir un mejor clasificador de documentos?

0. **Dataset más grande:** El conjunto de datos que usamos fue muy pequeño, considera usar los archivos corpus que estan ubicados en la ruta: `datasets/email/plaintext/`

1. **Limpieza:** como te diste cuenta no hicimos ningun tipo de limpieza de texto en los correos electrónicos. Considera usar expresiones regulares, filtros por categorias gramaticales, etc ... .

---

Con base en eso construye un dataset más grande y con un tokenizado más pulido.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# escribe tu código aquí:


2. **Validación del modelo anterior:**  
---

una vez tengas el nuevo conjunto de datos más pulido y de mayor tamaño, considera el mismo entrenamiento con el mismo tipo de atributos del ejemplo anterior, ¿mejora el accuracy del modelo resultante?

In [ ]:
# escribe tu código aquí:


3. **Construye mejores atributos**: A veces no solo se trata de las palabras más frecuentes sino de el contexto, y capturar contexto no es posible solo viendo los tokens de forma individual, ¿que tal si consideramos bi-gramas, tri-gramas ...?, ¿las secuencias de palabras podrián funcionar como mejores atributos para el modelo?. Para ver si es así,  podemos extraer n-gramas de nuestro corpus y obtener sus frecuencias de aparición con `FreqDist()`, desarrolla tu propia manera de hacerlo y entrena un modelo con esos nuevos atributos, no olvides compartir tus resultados en la sección de comentarios.

In [ ]:
# escribe tu código aquí:


In [ ]:
import math
import os

In [ ]:
os.listdir('corpus1/spam')

## Preparación del corpus de emails

In [ ]:
!git clone https://github.com/pachocamacho1990/datasets

Cloning into 'datasets'...
remote: Enumerating objects: 39, done.
remote: Total 39 (delta 0), reused 0 (delta 0), pack-reused 39
Unpacking objects: 100% (39/39), done.


In [ ]:
! unzip datasets/email/plaintext/corpus1.zip

In [ ]:
data = []
clases = []
#lectura de spam data
for file in os.listdir('corpus1/spam'):
  with open('corpus1/spam/'+file, encoding='latin-1') as f:
    data.append(f.read())
    clases.append('spam')
#lectura de ham data
for file in os.listdir('corpus1/ham'):
  with open('corpus1/ham/'+file, encoding='latin-1') as f:
    data.append(f.read())
    clases.append('ham')
len(data)

5172

## Construcción de modelo Naive Bayes

### Tokenizador de Spacy

* Documentación: https://spacy.io/api/tokenizer
* ¿Cómo funciona el tokenizador? https://spacy.io/usage/linguistic-features#how-tokenizer-works

In [ ]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English

nlp = English()
tokenizer = Tokenizer(nlp.vocab)

In [ ]:
print([t.text for t in tokenizer(data[0])])

['Subject:', 'brand', 'new', 'teenager', 'peeing', '\n', 'you', 'don', "'", 't', 'know', 'me', 'from', 'adam', '.', ':', ')', 'iyakubonana', '\n', 'nothing', 'that', 'you', 'have', 'not', 'given', 'away', 'will', 'ever', 'be', 'really', 'yours', '.', '\n', 'i', 'don', "'", 't', 'deserve', 'this', 'award', ',', 'but', 'i', 'have', 'arthritis', ',', 'and', 'i', 'don', "'", 't', 'deserve', 'that', ',', 'either', '.', 'the', 'most', 'important', 'thing', 'a', 'father', 'can', 'do', 'for', 'his', 'children', 'is', 'to', 'love', 'their', 'mother', '.', '\n', 'tricks', 'and', 'treachery', 'are', 'the', 'practice', 'of', 'fools', ',', 'that', 'don', "'", 't', 'have', 'brains', 'enough', 'to', 'be', 'honest', '.', '\n', 'a', 'heart', 'that', 'loves', 'is', 'always', 'young', '.', '\n', 'if', 'you', 'drink', ',', 'don', "'", 't', 'drive', '.', 'don', "'", 't', 'even', 'putt', '.', '\n', 'husbands', 'never', 'become', 'good', 'they', 'merely', 'become', 'proficient', '.', '\n', 'only', 'from', 't

### Clase principal para el algoritmo

Recuerda que la clase más probable viene dada por (en espacio de cómputo logarítmico):


$$\hat{c} = {\arg \max}_{(c)}\log{P(c)}
 +\sum_{i=1}^n
\log{ P(f_i \vert c)}
$$

Donde, para evitar casos atípicos, usaremos el suavizado de Laplace así:

$$
P(f_i \vert c) = \frac{C(f_i, c)+1}{C(c) + \vert V \vert}
$$

siendo $\vert V \vert$ la longitud del vocabulario de nuestro conjunto de entrenamiento.

In [ ]:
import numpy as np

class NaiveBayesClassifier():
  nlp = English()
  tokenizer = Tokenizer(nlp.vocab)

  def tokenize(self, doc):
    return  [t.text.lower() for t in tokenizer(doc)]

  def word_counts(self, words):
    wordCount = {}
    for w in words:
      if w in wordCount.keys():
        wordCount[w] += 1
      else:
        wordCount[w] = 1
    return wordCount

  def fit(self, data, clases):
    n = len(data)
    self.unique_clases = set(clases)
    self.vocab = set()
    self.classCount = {} #C(c)
    self.log_classPriorProb = {} #P(c)
    self.wordConditionalCounts = {} #C(w|c)
    #conteos de clases
    for c in clases:
      if c in self.classCount.keys():
        self.classCount[c] += 1
      else:
        self.classCount[c] = 1
    # calculo de P(c)
    for c in self.classCount.keys():
      self.log_classPriorProb[c] = math.log(self.classCount[c]/n)
      self.wordConditionalCounts[c] = {}
    # calculo de C(w|c)
    for text, c in zip(data, clases):
      counts = self.word_counts(self.tokenize(text))
      for word, count in counts.items():
        if word not in self.vocab:
          self.vocab.add(word)
        if word not in self.wordConditionalCounts[c]:
          self.wordConditionalCounts[c][word] = 0.0
        self.wordConditionalCounts[c][word] += count

  def predict(self, data):
    results = []
    for text in data:
      words = set(self.tokenize(text))
      scoreProb = {}
      for word in words:
        if word not in self.vocab: continue #ignoramos palabras nuevas
        #suavizado Laplaciano para P(w|c)
        for c in self.unique_clases:
          log_wordClassProb = math.log(
              (self.wordConditionalCounts[c].get(word, 0.0)+1)/(self.classCount[c]+len(self.vocab)))
          scoreProb[c] = scoreProb.get(c, self.log_classPriorProb[c]) + log_wordClassProb
      arg_maxprob = np.argmax(np.array(list(scoreProb.values())))
      results.append(list(scoreProb.keys())[arg_maxprob])
    return results


### Utilidades de Scikit Learn
* `train_test_split`: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

* `accuracy_score`: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html

* `precision_score`: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html

* `recall_score`: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [ ]:
data_train, data_test, clases_train, clases_test = train_test_split(data, clases, test_size=0.10, random_state=42)

In [ ]:
classifier = NaiveBayesClassifier()
classifier.fit(data_train, clases_train)

In [ ]:
clases_predict = classifier.predict(data_test)

In [ ]:
accuracy_score(clases_test, clases_predict)

0.8552123552123552

In [ ]:
precision_score(clases_test, clases_predict, average=None, zero_division=1)

array([0.82876712, 1.        ])

In [ ]:
recall_score(clases_test, clases_predict, average=None, zero_division=1)

array([1.        , 0.51612903])